In [1]:
%matplotlib inline
import numpy 
import pandas
import matplotlib.pyplot as plt
import seaborn as sns
import re

In [223]:
accounting_neeq = pandas.read_excel("../data/raw/accounting2015.xlsx")
accounting_cf_neeq = pandas.read_excel("../data/raw/accounting2015_cf.xlsx")
accounting_neeq = accounting_neeq[:-2]
accounting_cf_neeq = accounting_cf_neeq[:-2]
#833822

In [224]:
accounting_neeq = pandas.merge(accounting_neeq, accounting_cf_neeq, on=["证券代码", "证券简称"])

In [225]:
columns = ["资产总计", "归属母公司股东的权益", "营业收入", "营业成本", "销售费用", 
           "管理费用", "财务费用", "归属母公司股东的净利润", "非经常性损益", "扣除非经常性损益后的净利", 
          "经营活动产生的现金流量净额", "应收账款", "固定资产", "ROA", "折旧", "长期待摊费用摊销",
           "存货的减少", "经营性应收项目的减少", "经营性应付项目的增加"]

year = numpy.arange(2003, 2016)

column_names = [item + str(iyear) for item in columns for iyear in year]

In [226]:
accounting_neeq.columns = ["证券代码", "证券简称"] + column_names

In [227]:
accounting_neeq_long = pandas.wide_to_long(accounting_neeq, stubnames=columns, i="证券代码", j="年度")

In [228]:
neeq_sample = pandas.read_csv("../data/NEEQ_sample.csv", parse_dates=[7, 8], index_col=13)

In [229]:
neeq_sample = neeq_sample[["挂牌日期", "主办券商", "所属园区", "所属证监会行业", "摘牌日期"]]
neeq_sample = neeq_sample.sort_index()
neeq_sample["挂牌年度"] = neeq_sample["挂牌日期"].apply(lambda x: x.year)
neeq_sample["挂牌月份"] = neeq_sample["挂牌日期"].apply(lambda x: x.month)

In [230]:
accounting_neeq_long = pandas.merge(neeq_sample, accounting_neeq_long, 
                                    how="inner", left_index=True, 
                                    right_index=True)

In [231]:
accounting_neeq_long.reset_index(inplace=True)

In [232]:
accounting_neeq_long.head()

,证券代码,年度,挂牌日期,主办券商,所属园区,所属证监会行业,摘牌日期,挂牌年度,挂牌月份,证券简称,...,扣除非经常性损益后的净利,经营活动产生的现金流量净额,应收账款,固定资产,ROA,折旧,长期待摊费用摊销,存货的减少,经营性应收项目的减少,经营性应付项目的增加
0,430001.OC,2003,2006-01-18,申银万国证券股份有限公司,中关村科技园区非上市股份有限公司,软件与服务,2010-12-06,2006,1,世纪瑞尔(退市),...,NaN,10843347.75,27121681.98,29395524.57,NaN,943428.0,610643.0,-2692313.1,-13831571.1,13440729.88
1,430002.OC,2003,2006-01-23,申万宏源证券有限公司,中关村科技园区,软件和信息技术服务业,NaT,2006,1,中科软,...,6039888.09,NaN,19541521.43,6683984.09,NaN,NaN,NaN,NaN,NaN,NaN
2,430003.OC,2003,2006-03-31,国泰君安证券股份有限公司,中关村科技园区,通用设备制造业,NaT,2006,3,北京时代,...,NaN,NaN,8586266.12,51466641.87,NaN,NaN,NaN,NaN,NaN,NaN
3,430004.OC,2003,2006-06-07,广发证券股份有限公司,中关村科技园区,专用设备制造业,NaT,2006,6,绿创环保,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,430005.OC,2003,2006-07-28,广发证券股份有限公司,中关村科技园区,医药制造业,NaT,2006,7,原子高科,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [233]:
files = !ls ../data/raw/
files_fin_state = []
for file in files:
    match = re.match("财报披露日", file)
    if match is not None:
        files_fin_state.append(file)

In [234]:
fin_report_data = []
for item in files_fin_state:
    data_dir = "../data/raw/" + item
    data = pandas.read_excel(data_dir)
    data = data[:-2]
    fin_report_data.append(data)

In [235]:
financial_reports = pandas.concat(fin_report_data)

In [236]:
financial_reports = financial_reports[["证券代码", "实际披露日期", "报告期"]]
financial_reports["年度"] = financial_reports["报告期"].apply(lambda x: x.year)

In [237]:
financial_reports.head()

,证券代码,实际披露日期,报告期,年度
0,400028.OC,2004-05-19,2003-12-31,2003
1,400029.OC,2004-04-22,2003-12-31,2003
2,400030.OC,2004-04-17,2003-12-31,2003
3,400031.OC,2004-03-19,2003-12-31,2003
4,400032.OC,2004-04-29,2003-12-31,2003


In [238]:
accounting_neeq_long = pandas.merge(accounting_neeq_long, financial_reports, 
                                    how="left", on=["证券代码", "年度"])

In [239]:
accounting_neeq_long.iloc[0]

证券代码                       430001.OC
年度                              2003
挂牌日期             2006-01-18 00:00:00
主办券商                    申银万国证券股份有限公司
所属园区                中关村科技园区非上市股份有限公司
所属证监会行业                        软件与服务
摘牌日期             2010-12-06 00:00:00
挂牌年度                            2006
挂牌月份                               1
证券简称                        世纪瑞尔(退市)
资产总计                     1.03696e+08
归属母公司股东的权益               7.06015e+07
营业收入                     4.02414e+07
营业成本                     1.35694e+07
销售费用                     7.71494e+06
管理费用                     9.77092e+06
财务费用                          235899
归属母公司股东的净利润               1.0645e+07
非经常性损益                           NaN
扣除非经常性损益后的净利                     NaN
经营活动产生的现金流量净额            1.08433e+07
应收账款                     2.71217e+07
固定资产                     2.93955e+07
ROA                              NaN
折旧                            943428
长期待摊费用摊销                      610643
存货的减少                   -2.69231e+06
经

In [240]:
accounting_neeq_long["挂牌前后"] = numpy.nan

In [241]:
accounting_neeq_long.loc[(accounting_neeq_long["年度"] >= accounting_neeq_long["挂牌年度"]), "挂牌前后"] = "挂牌后"
accounting_neeq_long.loc[(accounting_neeq_long["年度"] == accounting_neeq_long["挂牌年度"] - 1) &
                         (accounting_neeq_long["实际披露日期"] > accounting_neeq_long["挂牌日期"]), "挂牌前后"] = "挂牌后"

accounting_neeq_long.loc[(accounting_neeq_long["年度"] <= accounting_neeq_long["挂牌年度"] - 2), "挂牌前后"] = "挂牌前"
accounting_neeq_long.loc[(accounting_neeq_long["年度"] == accounting_neeq_long["挂牌年度"] - 1) &
                         (accounting_neeq_long["实际披露日期"] <= accounting_neeq_long["挂牌日期"]), "挂牌前后"] = "挂牌前"

In [242]:
drops = accounting_neeq_long.loc[(accounting_neeq_long["年度"] < accounting_neeq_long["挂牌年度"] - 1) &
                                 (accounting_neeq_long.iloc[:, 10:28].isnull().all(axis=1))].index

In [243]:
accounting_neeq_long = accounting_neeq_long.drop(drops)

In [244]:
accounting_neeq_long.loc[(accounting_neeq_long["年度"] == accounting_neeq_long["挂牌年度"] - 1) &
                         (accounting_neeq_long["挂牌月份"] < 5), "挂牌前后"] = "挂牌后"

accounting_neeq_long.loc[(accounting_neeq_long["年度"] == accounting_neeq_long["挂牌年度"] - 1) &
                         (accounting_neeq_long["挂牌月份"] >= 5) & 
                         (~accounting_neeq_long.iloc[:, 10:28].isnull().all(axis=1)), "挂牌前后"] = "挂牌前"

In [245]:
accounting_neeq_long.loc[(accounting_neeq_long["证券代码"] == "832668.OC"), "挂牌前后"] = "挂牌后"

In [246]:
accounting_groups = accounting_neeq_long.groupby("证券代码")

In [247]:
def relative_index(data):
    data = data.sort_values("年度")
    len1 = len(data[data["挂牌前后"] == "挂牌前"])
    len2 = len(data[data["挂牌前后"] == "挂牌后"])
    year_pre = numpy.arange(start=-1 * len1, stop=0, step=1)
    year_post = numpy.arange(start=0, stop=len2, step=1)
    data["relativeYear"] = numpy.concatenate((year_pre, year_post))
    return data

In [248]:
accounting_neeq_long1 = accounting_groups.apply(relative_index)

In [249]:
accounting_neeq_long1 = accounting_neeq_long1.reset_index(drop=True)

In [250]:
accounting_neeq_long1[accounting_neeq_long1["relativeYear"] == -3].to_csv('../data/tmp2.csv', index=False)

In [417]:
accounting_neeq_long1.loc[accounting_neeq_long1["relativeYear"] == -1][~(accounting_neeq_long1.loc[accounting_neeq_long1["relativeYear"] == -1, "证券代码"].isin(
accounting_neeq_long1.loc[accounting_neeq_long1["relativeYear"] == -2, "证券代码"]))].to_csv("../data/tmp5.csv", index=False)

In [251]:
accounting_neeq_long1[accounting_neeq_long1["证券代码"]=="832608.OC"]

,证券代码,年度,挂牌日期,主办券商,所属园区,所属证监会行业,摘牌日期,挂牌年度,挂牌月份,证券简称,...,ROA,折旧,长期待摊费用摊销,存货的减少,经营性应收项目的减少,经营性应付项目的增加,实际披露日期,报告期,挂牌前后,relativeYear
11906,832608.OC,2012,2015-06-17,华龙证券股份有限公司,非高新产业区,软件和信息技术服务业,NaT,2015,6,天禹星,...,13.6161,19935709.42,521353.47,386376.70,14912773.25,-31105037.23,NaT,NaT,挂牌前,-3
11907,832608.OC,2013,2015-06-17,华龙证券股份有限公司,非高新产业区,软件和信息技术服务业,NaT,2015,6,天禹星,...,5.7529,17066122.53,544140.92,-1000688.04,-32042098.60,-6493504.34,NaT,NaT,挂牌前,-2
11908,832608.OC,2014,2015-06-17,华龙证券股份有限公司,非高新产业区,软件和信息技术服务业,NaT,2015,6,天禹星,...,-5.0298,18498351.34,834494.35,870360.17,13000704.04,-3661172.76,NaT,NaT,挂牌前,-1
11909,832608.OC,2015,2015-06-17,华龙证券股份有限公司,非高新产业区,软件和信息技术服务业,NaT,2015,6,天禹星,...,1.9566,18204453.91,1155913.54,-115187.29,-22449796.71,-2492503.10,2016-04-15,2015-12-31,挂牌后,0


In [252]:
StkcdChangedYear = ["832482.OC", "832498.OC", "832537.OC", "430702.OC", "830767.OC", "830780.OC", 
                   "830825.OC", "832383.OC", "832611.OC", "832615.OC", "832686.OC", "430213.OC",
                   "430214.OC", "430215.OC", "430217.OC", "430218.OC", "430219.OC", "430699.OC", 
                   "430720.OC", "430722.OC", "430724.OC", "430725.OC", "430727.OC", "430729.OC", 
                   "430730.OC", "430733.OC", "430734.OC", "430735.OC", "430736.OC", "430741.OC", 
                   "430743.OC", "430744.OC", "430745.OC", "430746.OC", "430747.OC", "430749.OC", 
                   "430750.OC", "430751.OC", "430752.OC", "430754.OC", "430755.OC", "430756.OC", 
                   "430757.OC", "430758.OC", "430759.OC", "430760.OC", "430762.OC", "430763.OC", 
                   "830773.OC", "830775.OC", "830778.OC", "830781.OC", "830785.OC", "830787.OC", 
                   "830790.OC", "830799.OC", "830800.OC", "830801.OC", "830805.OC", "830806.OC", 
                   "830819.OC", "832004.OC", "832066.OC", "832337.OC", "832347.OC", "832355.OC", 
                   "832360.OC", "832373.OC", "832374.OC", "832376.OC", "832377.OC", "832385.OC", 
                   "832386.OC", "832387.OC", "832388.OC", "832389.OC", "832390.OC", "832391.OC", 
                   "832392.OC", "832393.OC", "832394.OC", "832395.OC", "832398.OC", "832400.OC", 
                   "832401.OC", "832402.OC", "832403.OC", "832404.OC", "832406.OC", "832407.OC", 
                   "832408.OC", "832409.OC", "832410.OC", "832411.OC", "832412.OC", "832413.OC", 
                   "832414.OC", "832415.OC", "832416.OC", "832417.OC", "832418.OC", "832420.OC", 
                   "832421.OC", "832422.OC", "832423.OC", "832424.OC", "832425.OC", "832426.OC", 
                   "832427.OC", "832432.OC", "832433.OC", "832434.OC", "832436.OC", "832437.OC", 
                   "832438.OC", "832439.OC", "832440.OC", "832441.OC", "832442.OC", "832443.OC", 
                   "832444.OC", "832445.OC", "832446.OC", "832447.OC", "832448.OC", "832449.OC", 
                   "832452.OC", "832453.OC", "832454.OC", "832455.OC", "832456.OC", "832457.OC", 
                   "832461.OC", "832462.OC", "832463.OC", "832464.OC", "832465.OC", "832466.OC", 
                   "832467.OC", "832468.OC", "832469.OC", "832470.OC", "832471.OC", "832472.OC", 
                   "832473.OC", "832475.OC", "832477.OC", "832478.OC", "832479.OC", "832480.OC", 
                   "832481.OC", "832483.OC", "832484.OC", "832485.OC", "832486.OC", "832487.OC", 
                   "832488.OC", "832489.OC", "832490.OC", "832491.OC", "832492.OC", "832493.OC", 
                   "832494.OC", "832495.OC", "832496.OC", "832497.OC", "832500.OC", "832501.OC", 
                   "832502.OC", "832503.OC", "832504.OC", "832506.OC", "832507.OC", 
                   "832508.OC", "832509.OC", "832510.OC", "832515.OC", "832516.OC", "832517.OC", 
                   "832518.OC", "832520.OC", "832521.OC", "832522.OC", "832523.OC", "832524.PC", 
                   "832525.OC", "832526.OC", "832527.OC", "832529.OC", "832530.OC", "832531.OC", 
                   "832532.OC", "832534.OC", "832536.OC", "832538.OC", "832540.OC", "832542.OC", 
                   "832545.OC", "832546.OC", "832547.OC", "832552.OC", "832553.OC", "832556.OC", 
                   "832558.OC", "832559.OC", "832560.OC", "832561.OC", "832562.OC", "832563.OC", 
                   "832565.OC", "832566.OC", "832569.OC", "832578.OC", "832587.OC", "832588.OC", 
                   "832589.OC", "832592.OC", "832593.OC", "832594.OC", "832595.OC", "832596.OC", 
                   "832601.OC", "832606.OC", "832609.OC", "832610.OC", "832612.OC",
                   "832613.OC", "832614.OC", "832616.OC", "832629.OC", "832631.OC", "832634.OC", 
                   "832637.OC", "832639.OC", "832650.OC", "832663.OC", "832680.OC", "832687.OC", 
                   "832688.OC", "832753.OC", "832754.OC", "832773.OC", "832775.OC", "832778.OC",
                   "832779.OC", "832809.OC"]

In [253]:
for stkcd in StkcdChangedYear:
    accounting_neeq_long1.loc[accounting_neeq_long1["证券代码"] == stkcd, "relativeYear"] =\
    accounting_neeq_long1.loc[accounting_neeq_long1["证券代码"] == stkcd, "relativeYear"] + 1

In [254]:
StkcdChangedYear1 = ["430709.OC", "832372.OC", "832545.OC", "832548.OC", "832560.OC", "832601.OC", 
                     "836715.OC", "836910.OC"]

for stkcd in StkcdChangedYear1:
    accounting_neeq_long1.loc[accounting_neeq_long1["证券代码"] == stkcd, "relativeYear"] =\
    accounting_neeq_long1.loc[accounting_neeq_long1["证券代码"] == stkcd, "relativeYear"] - 1

In [32]:
StkcdAbsense = ["430072.OC", "834609.OC"]

In [255]:
accounting_neeq_long1.loc[(accounting_neeq_long1["证券代码"] == "830805.OC") & 
                          (accounting_neeq_long1["年度"] == 2013), 
                          ["实际披露日期", "报告期"]] = [pandas.datetime(2014, 6, 26), pandas.datetime(2013, 12, 31)]

In [256]:
accounting_neeq_long1.loc[(accounting_neeq_long1["证券代码"] == "830806.OC") & 
                          (accounting_neeq_long1["年度"] == 2013), 
                          ["实际披露日期", "报告期"]] = [pandas.datetime(2014, 9, 24), pandas.datetime(2013, 12, 31)]

In [257]:
accounting_neeq_long1.loc[(accounting_neeq_long1["证券代码"] == "832385.OC") & 
                          (accounting_neeq_long1["年度"] == 2014), 
                          ["实际披露日期", "报告期"]] = [pandas.datetime(2015, 5, 20), pandas.datetime(2014, 12, 31)]

In [258]:
accounting_neeq_long1.loc[(accounting_neeq_long1["证券代码"] == "832809.OC") & 
                          (accounting_neeq_long1["年度"] == 2014), 
                          ["实际披露日期", "报告期"]] = [pandas.datetime(2015, 8, 21), pandas.datetime(2014, 12, 31)]

In [259]:
accounting_neeq_long1.loc[(accounting_neeq_long1["证券代码"] == "832779.OC") & 
                          (accounting_neeq_long1["年度"] == 2014), 
                          "报告期"] = pandas.datetime(2014, 12, 31)

In [260]:
accounting_neeq_long1.loc[(accounting_neeq_long1["证券代码"] == "832778.OC") & 
                          (accounting_neeq_long1["年度"] == 2014), 
                          ["实际披露日期", "报告期"]] = [pandas.datetime(2015, 7, 20), pandas.datetime(2014, 12, 31)]

In [261]:
accounting_neeq_long1.loc[(accounting_neeq_long1["证券代码"] == "832775.OC") & 
                          (accounting_neeq_long1["年度"] == 2014), 
                          ["实际披露日期", "报告期"]] = [pandas.datetime(2015, 8, 19), pandas.datetime(2014, 12, 31)]

In [262]:
accounting_neeq_long1.loc[(accounting_neeq_long1["证券代码"] == "832773.OC") & 
                          (accounting_neeq_long1["年度"] == 2014), 
                          ["实际披露日期", "报告期"]] = [pandas.datetime(2015, 7, 24), pandas.datetime(2014, 12, 31)]

In [263]:
accounting_neeq_long1.loc[(accounting_neeq_long1["证券代码"] == "832754.OC") & 
                          (accounting_neeq_long1["年度"] == 2014), 
                          ["实际披露日期", "报告期"]] = [pandas.datetime(2015, 8, 5), pandas.datetime(2014, 12, 31)]

In [264]:
accounting_neeq_long1.loc[(accounting_neeq_long1["证券代码"] == "832753.OC") & 
                          (accounting_neeq_long1["年度"] == 2014), 
                          ["实际披露日期", "报告期"]] = [pandas.datetime(2015, 8, 13), pandas.datetime(2014, 12, 31)]

In [265]:
accounting_neeq_long1.loc[(accounting_neeq_long1["证券代码"] == "832688.OC") & 
                          (accounting_neeq_long1["年度"] == 2014), 
                          ["实际披露日期", "报告期"]] = [pandas.datetime(2015, 7, 13), pandas.datetime(2014, 12, 31)]

In [266]:
accounting_neeq_long1.loc[(accounting_neeq_long1["证券代码"] == "832687.OC") & 
                          (accounting_neeq_long1["年度"] == 2014), 
                          ["实际披露日期", "报告期"]] = [pandas.datetime(2015, 7, 28), pandas.datetime(2014, 12, 31)]

In [267]:
accounting_neeq_long1.loc[(accounting_neeq_long1["证券代码"] == "832680.OC") & 
                          (accounting_neeq_long1["年度"] == 2014), 
                          ["实际披露日期", "报告期"]] = [pandas.datetime(2015, 6, 29), pandas.datetime(2014, 12, 31)]

In [268]:
accounting_neeq_long1.loc[(accounting_neeq_long1["证券代码"] == "832663.OC") & 
                          (accounting_neeq_long1["年度"] == 2014), 
                          "报告期"] = pandas.datetime(2014, 12, 31)

In [269]:
accounting_neeq_long1.loc[(accounting_neeq_long1["证券代码"] == "832650.OC") & 
                          (accounting_neeq_long1["年度"] == 2014), 
                          ["实际披露日期", "报告期"]] = [pandas.datetime(2015, 7, 15), pandas.datetime(2014, 12, 31)]

In [270]:
accounting_neeq_long1.loc[(accounting_neeq_long1["证券代码"] == "832639.OC") & 
                          (accounting_neeq_long1["年度"] == 2014), 
                          ["实际披露日期", "报告期"]] = [pandas.datetime(2015, 6, 16), pandas.datetime(2014, 12, 31)]

In [271]:
accounting_neeq_long1.loc[(accounting_neeq_long1["证券代码"] == "832637.OC") & 
                          (accounting_neeq_long1["年度"] == 2014), 
                          ["实际披露日期", "报告期"]] = [pandas.datetime(2015, 7, 27), pandas.datetime(2014, 12, 31)]

In [272]:
accounting_neeq_long1.loc[(accounting_neeq_long1["证券代码"] == "832634.OC") & 
                          (accounting_neeq_long1["年度"] == 2014), 
                          ["实际披露日期", "报告期"]] = [pandas.datetime(2015, 6, 16), pandas.datetime(2014, 12, 31)]

In [273]:
accounting_neeq_long1.loc[(accounting_neeq_long1["证券代码"] == "832631.OC") & 
                          (accounting_neeq_long1["年度"] == 2014), 
                          ["实际披露日期", "报告期"]] = [pandas.datetime(2015, 6, 16), pandas.datetime(2014, 12, 31)]

In [274]:
accounting_neeq_long1.loc[(accounting_neeq_long1["证券代码"] == "832629.OC") & 
                          (accounting_neeq_long1["年度"] == 2014), 
                          ["实际披露日期", "报告期"]] = [pandas.datetime(2015, 7, 7), pandas.datetime(2014, 12, 31)]

In [275]:
accounting_neeq_long1.loc[(accounting_neeq_long1["证券代码"] == "832616.OC") & 
                          (accounting_neeq_long1["年度"] == 2014), 
                          ["实际披露日期", "报告期"]] = [pandas.datetime(2015, 7, 10), pandas.datetime(2014, 12, 31)]

In [276]:
accounting_neeq_long1.loc[(accounting_neeq_long1["证券代码"] == "832614.OC") & 
                          (accounting_neeq_long1["年度"] == 2014), 
                          ["实际披露日期", "报告期"]] = [pandas.datetime(2015, 6, 11), pandas.datetime(2014, 12, 31)]

In [277]:
accounting_neeq_long1.loc[(accounting_neeq_long1["证券代码"] == "832613.OC") & 
                          (accounting_neeq_long1["年度"] == 2014), 
                          ["实际披露日期", "报告期"]] = [pandas.datetime(2015, 6, 9), pandas.datetime(2014, 12, 31)]

In [278]:
accounting_neeq_long1.loc[(accounting_neeq_long1["证券代码"] == "832612.OC") & 
                          (accounting_neeq_long1["年度"] == 2014), 
                          ["实际披露日期", "报告期"]] = [pandas.datetime(2015, 6, 24), pandas.datetime(2014, 12, 31)]

In [279]:
accounting_neeq_long1.loc[(accounting_neeq_long1["证券代码"] == "832610.OC") & 
                          (accounting_neeq_long1["年度"] == 2014), 
                          ["实际披露日期", "报告期"]] = [pandas.datetime(2015, 7, 2), pandas.datetime(2014, 12, 31)]

In [280]:
accounting_neeq_long1.loc[(accounting_neeq_long1["证券代码"] == "832609.OC") & 
                          (accounting_neeq_long1["年度"] == 2014), 
                          ["实际披露日期", "报告期"]] = [pandas.datetime(2015, 6, 30), pandas.datetime(2014, 12, 31)]

In [281]:
accounting_neeq_long1.loc[(accounting_neeq_long1["证券代码"] == "832606.OC") & 
                          (accounting_neeq_long1["年度"] == 2014), 
                          ["实际披露日期", "报告期"]] = [pandas.datetime(2015, 6, 10), pandas.datetime(2014, 12, 31)]

In [282]:
accounting_neeq_long1.loc[(accounting_neeq_long1["证券代码"] == "832601.OC") & 
                          (accounting_neeq_long1["年度"] == 2014), 
                          "报告期"] = pandas.datetime(2014, 12, 31)

In [283]:
accounting_neeq_long1.loc[(accounting_neeq_long1["证券代码"] == "832596.OC") & 
                          (accounting_neeq_long1["年度"] == 2014), 
                          ["实际披露日期", "报告期"]] = [pandas.datetime(2015, 7, 3), pandas.datetime(2014, 12, 31)]

In [284]:
accounting_neeq_long1.loc[(accounting_neeq_long1["证券代码"] == "832595.OC") & 
                          (accounting_neeq_long1["年度"] == 2014), 
                          ["实际披露日期", "报告期"]] = [pandas.datetime(2015, 7, 3), pandas.datetime(2014, 12, 31)]

In [285]:
accounting_neeq_long1.loc[(accounting_neeq_long1["证券代码"] == "832594.OC") & 
                          (accounting_neeq_long1["年度"] == 2014), 
                          ["实际披露日期", "报告期"]] = [pandas.datetime(2015, 7, 8), pandas.datetime(2014, 12, 31)]

In [286]:
accounting_neeq_long1.loc[(accounting_neeq_long1["证券代码"] == "832593.OC") & 
                          (accounting_neeq_long1["年度"] == 2014), 
                          ["实际披露日期", "报告期"]] = [pandas.datetime(2015, 6, 12), pandas.datetime(2014, 12, 31)]

In [287]:
accounting_neeq_long1.loc[(accounting_neeq_long1["证券代码"] == "832592.OC") & 
                          (accounting_neeq_long1["年度"] == 2014), 
                          "报告期"] = pandas.datetime(2014, 12, 31)

In [288]:
accounting_neeq_long1.loc[(accounting_neeq_long1["证券代码"] == "832589.OC") & 
                          (accounting_neeq_long1["年度"] == 2014), 
                          ["实际披露日期", "报告期"]] = [pandas.datetime(2015, 6, 11), pandas.datetime(2014, 12, 31)]

In [289]:
accounting_neeq_long1.loc[(accounting_neeq_long1["证券代码"] == "832588.OC") & 
                          (accounting_neeq_long1["年度"] == 2014), 
                          "报告期"] = pandas.datetime(2014, 12, 31)

In [290]:
accounting_neeq_long1.loc[(accounting_neeq_long1["证券代码"] == "832587.OC") & 
                          (accounting_neeq_long1["年度"] == 2014), 
                          "报告期"] = pandas.datetime(2014, 12, 31)

In [291]:
accounting_neeq_long1.loc[(accounting_neeq_long1["证券代码"] == "832578.OC") & 
                          (accounting_neeq_long1["年度"] == 2014), 
                          ["实际披露日期", "报告期"]] = [pandas.datetime(2015, 6, 29), pandas.datetime(2014, 12, 31)]

In [292]:
accounting_neeq_long1.loc[(accounting_neeq_long1["证券代码"] == "832569.OC") & 
                          (accounting_neeq_long1["年度"] == 2014), 
                          ["实际披露日期", "报告期"]] = [pandas.datetime(2015, 6, 10), pandas.datetime(2014, 12, 31)]

In [293]:
accounting_neeq_long1.loc[(accounting_neeq_long1["证券代码"] == "832566.OC") & 
                          (accounting_neeq_long1["年度"] == 2014), 
                          ["实际披露日期", "报告期"]] = [pandas.datetime(2015, 7, 7), pandas.datetime(2014, 12, 31)]

In [294]:
accounting_neeq_long1.loc[(accounting_neeq_long1["证券代码"] == "832565.OC") & 
                          (accounting_neeq_long1["年度"] == 2014), 
                          "报告期"] = pandas.datetime(2014, 12, 31)

In [295]:
accounting_neeq_long1.loc[(accounting_neeq_long1["证券代码"] == "832563.OC") & 
                          (accounting_neeq_long1["年度"] == 2014), 
                          ["实际披露日期", "报告期"]] = [pandas.datetime(2015, 8, 4), pandas.datetime(2014, 12, 31)]

In [296]:
accounting_neeq_long1.loc[(accounting_neeq_long1["证券代码"] == "832562.OC") & 
                          (accounting_neeq_long1["年度"] == 2014), 
                          ["实际披露日期", "报告期"]] = [pandas.datetime(2015, 7, 1), pandas.datetime(2014, 12, 31)]

In [297]:
accounting_neeq_long1.loc[(accounting_neeq_long1["证券代码"] == "832561.OC") & 
                          (accounting_neeq_long1["年度"] == 2014), 
                          "报告期"] = pandas.datetime(2014, 12, 31)

In [298]:
accounting_neeq_long1.loc[(accounting_neeq_long1["证券代码"] == "832560.OC") & 
                          (accounting_neeq_long1["年度"] == 2014), 
                          ["实际披露日期", "报告期"]] = [pandas.datetime(2015, 6, 30), pandas.datetime(2014, 12, 31)]

In [299]:
accounting_neeq_long1.loc[(accounting_neeq_long1["证券代码"] == "832559.OC") & 
                          (accounting_neeq_long1["年度"] == 2014), 
                          ["实际披露日期", "报告期"]] = [pandas.datetime(2015, 7, 2), pandas.datetime(2014, 12, 31)]

In [300]:
accounting_neeq_long1.loc[(accounting_neeq_long1["证券代码"] == "832558.OC") & 
                          (accounting_neeq_long1["年度"] == 2014), 
                          ["实际披露日期", "报告期"]] = [pandas.datetime(2015, 6, 12), pandas.datetime(2014, 12, 31)]

In [301]:
accounting_neeq_long1.loc[(accounting_neeq_long1["证券代码"] == "832556.OC") & 
                          (accounting_neeq_long1["年度"] == 2014), 
                          ["实际披露日期", "报告期"]] = [pandas.datetime(2015, 6, 11), pandas.datetime(2014, 12, 31)]

In [302]:
accounting_neeq_long1.loc[(accounting_neeq_long1["证券代码"] == "832553.OC") & 
                          (accounting_neeq_long1["年度"] == 2014), 
                          ["实际披露日期", "报告期"]] = [pandas.datetime(2015, 6, 5), pandas.datetime(2014, 12, 31)]

In [303]:
accounting_neeq_long1.loc[(accounting_neeq_long1["证券代码"] == "832552.OC") & 
                          (accounting_neeq_long1["年度"] == 2014), 
                          ["实际披露日期", "报告期"]] = [pandas.datetime(2015, 6, 16), pandas.datetime(2014, 12, 31)]

In [304]:
accounting_neeq_long1.loc[(accounting_neeq_long1["证券代码"] == "832548.OC") & 
                          (accounting_neeq_long1["年度"] == 2014), 
                          ["实际披露日期", "报告期"]] = [pandas.datetime(2015, 7, 7), pandas.datetime(2014, 12, 31)]

In [305]:
accounting_neeq_long1.loc[(accounting_neeq_long1["证券代码"] == "832547.OC") & 
                          (accounting_neeq_long1["年度"] == 2014), 
                          ["实际披露日期", "报告期"]] = [pandas.datetime(2015, 6, 4), pandas.datetime(2014, 12, 31)]

In [306]:
accounting_neeq_long1.loc[(accounting_neeq_long1["证券代码"] == "832546.OC") & 
                          (accounting_neeq_long1["年度"] == 2014), 
                          ["实际披露日期", "报告期"]] = [pandas.datetime(2015, 6, 24), pandas.datetime(2014, 12, 31)]

In [307]:
accounting_neeq_long1.loc[(accounting_neeq_long1["证券代码"] == "832545.OC") & 
                          (accounting_neeq_long1["年度"] == 2014), 
                          ["实际披露日期", "报告期"]] = [pandas.datetime(2015, 7, 2), pandas.datetime(2014, 12, 31)]

In [308]:
accounting_neeq_long1.loc[(accounting_neeq_long1["证券代码"] == "832542.OC") & 
                          (accounting_neeq_long1["年度"] == 2014), 
                          ["实际披露日期", "报告期"]] = [pandas.datetime(2015, 6, 24), pandas.datetime(2014, 12, 31)]

In [309]:
accounting_neeq_long1.loc[(accounting_neeq_long1["证券代码"] == "832540.OC") & 
                          (accounting_neeq_long1["年度"] == 2014), 
                          ["实际披露日期", "报告期"]] = [pandas.datetime(2015, 6, 5), pandas.datetime(2014, 12, 31)]

In [310]:
accounting_neeq_long1.loc[(accounting_neeq_long1["证券代码"] == "832538.OC") & 
                          (accounting_neeq_long1["年度"] == 2014), 
                          ["实际披露日期", "报告期"]] = [pandas.datetime(2015, 6, 8), pandas.datetime(2014, 12, 31)]

In [311]:
accounting_neeq_long1.loc[(accounting_neeq_long1["证券代码"] == "832536.OC") & 
                          (accounting_neeq_long1["年度"] == 2014), 
                          ["实际披露日期", "报告期"]] = [pandas.datetime(2015, 8, 6), pandas.datetime(2014, 12, 31)]

In [312]:
accounting_neeq_long1.loc[(accounting_neeq_long1["证券代码"] == "832532.OC") & 
                          (accounting_neeq_long1["年度"] == 2014), 
                          ["实际披露日期", "报告期"]] = [pandas.datetime(2015, 6, 12), pandas.datetime(2014, 12, 31)]

In [313]:
accounting_neeq_long1.loc[(accounting_neeq_long1["证券代码"] == "832531.OC") & 
                          (accounting_neeq_long1["年度"] == 2014), 
                          ["实际披露日期", "报告期"]] = [pandas.datetime(2015, 6, 8), pandas.datetime(2014, 12, 31)]

In [314]:
accounting_neeq_long1.loc[(accounting_neeq_long1["证券代码"] == "832530.OC") & 
                          (accounting_neeq_long1["年度"] == 2014), 
                          ["实际披露日期", "报告期"]] = [pandas.datetime(2015, 7, 9), pandas.datetime(2014, 12, 31)]

In [315]:
accounting_neeq_long1.loc[(accounting_neeq_long1["证券代码"] == "832529.OC") & 
                          (accounting_neeq_long1["年度"] == 2014), 
                          "报告期"] = pandas.datetime(2014, 12, 31)

In [316]:
accounting_neeq_long1.loc[(accounting_neeq_long1["证券代码"] == "832526.OC") & 
                          (accounting_neeq_long1["年度"] == 2014), 
                          ["实际披露日期", "报告期"]] = [pandas.datetime(2015, 6, 15), pandas.datetime(2014, 12, 31)]

In [317]:
accounting_neeq_long1.loc[(accounting_neeq_long1["证券代码"] == "832525.OC") & 
                          (accounting_neeq_long1["年度"] == 2014), 
                          ["实际披露日期", "报告期"]] = [pandas.datetime(2015, 6, 11), pandas.datetime(2014, 12, 31)]

In [318]:
accounting_neeq_long1.loc[(accounting_neeq_long1["证券代码"] == "832524.OC") & 
                          (accounting_neeq_long1["年度"] == 2014), 
                          ["实际披露日期", "报告期"]] = [pandas.datetime(2015, 7, 3), pandas.datetime(2014, 12, 31)]

In [319]:
accounting_neeq_long1.loc[(accounting_neeq_long1["证券代码"] == "832521.OC") & 
                          (accounting_neeq_long1["年度"] == 2014), 
                          ["实际披露日期", "报告期"]] = [pandas.datetime(2015, 6, 9), pandas.datetime(2014, 12, 31)]

In [320]:
accounting_neeq_long1.loc[(accounting_neeq_long1["证券代码"] == "832522.OC") & 
                          (accounting_neeq_long1["年度"] == 2014), 
                          ["实际披露日期", "报告期"]] = [pandas.datetime(2015, 6, 8), pandas.datetime(2014, 12, 31)]

In [321]:
accounting_neeq_long1.loc[(accounting_neeq_long1["证券代码"] == "832520.OC") & 
                          (accounting_neeq_long1["年度"] == 2014), 
                          ["实际披露日期", "报告期"]] = [pandas.datetime(2015, 5, 28), pandas.datetime(2014, 12, 31)]

In [322]:
accounting_neeq_long1.loc[(accounting_neeq_long1["证券代码"] == "832518.OC") & 
                          (accounting_neeq_long1["年度"] == 2014), 
                          ["实际披露日期", "报告期"]] = [pandas.datetime(2015, 5, 29), pandas.datetime(2014, 12, 31)]

In [323]:
accounting_neeq_long1.loc[(accounting_neeq_long1["证券代码"] == "832517.OC") & 
                          (accounting_neeq_long1["年度"] == 2014), 
                          ["实际披露日期", "报告期"]] = [pandas.datetime(2015, 5, 27), pandas.datetime(2014, 12, 31)]

In [324]:
accounting_neeq_long1.loc[(accounting_neeq_long1["证券代码"] == "832516.OC") & 
                          (accounting_neeq_long1["年度"] == 2014), 
                          ["实际披露日期", "报告期"]] = [pandas.datetime(2015, 6, 9), pandas.datetime(2014, 12, 31)]

In [325]:
accounting_neeq_long1.loc[(accounting_neeq_long1["证券代码"] == "832515.OC") & 
                          (accounting_neeq_long1["年度"] == 2014), 
                          ["实际披露日期", "报告期"]] = [pandas.datetime(2015, 6, 29), pandas.datetime(2014, 12, 31)]

In [326]:
accounting_neeq_long1.loc[(accounting_neeq_long1["证券代码"] == "832510.OC") & 
                          (accounting_neeq_long1["年度"] == 2014), 
                          ["实际披露日期", "报告期"]] = [pandas.datetime(2015, 6, 10), pandas.datetime(2014, 12, 31)]

In [327]:
accounting_neeq_long1.loc[(accounting_neeq_long1["证券代码"] == "832509.OC") & 
                          (accounting_neeq_long1["年度"] == 2014), 
                          ["实际披露日期", "报告期"]] = [pandas.datetime(2015, 5, 29), pandas.datetime(2014, 12, 31)]

In [328]:
accounting_neeq_long1.loc[(accounting_neeq_long1["证券代码"] == "832508.OC") & 
                          (accounting_neeq_long1["年度"] == 2014), 
                          ["实际披露日期", "报告期"]] = [pandas.datetime(2015, 6, 12), pandas.datetime(2014, 12, 31)]

In [329]:
accounting_neeq_long1.loc[(accounting_neeq_long1["证券代码"] == "832507.OC") & 
                          (accounting_neeq_long1["年度"] == 2014), 
                          ["实际披露日期", "报告期"]] = [pandas.datetime(2015, 5, 22), pandas.datetime(2014, 12, 31)]

In [330]:
accounting_neeq_long1.loc[(accounting_neeq_long1["证券代码"] == "832506.OC") & 
                          (accounting_neeq_long1["年度"] == 2014), 
                          ["实际披露日期", "报告期"]] = [pandas.datetime(2015, 6, 8), pandas.datetime(2014, 12, 31)]

In [331]:
accounting_neeq_long1.loc[(accounting_neeq_long1["证券代码"] == "832502.OC") & 
                          (accounting_neeq_long1["年度"] == 2014), 
                          ["实际披露日期", "报告期"]] = [pandas.datetime(2015, 6, 12), pandas.datetime(2014, 12, 31)]

In [332]:
accounting_neeq_long1.loc[(accounting_neeq_long1["证券代码"] == "832503.OC") & 
                          (accounting_neeq_long1["年度"] == 2014), 
                          ["实际披露日期", "报告期"]] = [pandas.datetime(2015, 5, 28), pandas.datetime(2014, 12, 31)]

In [333]:
accounting_neeq_long1.loc[(accounting_neeq_long1["证券代码"] == "832501.OC") & 
                          (accounting_neeq_long1["年度"] == 2014), 
                          ["实际披露日期", "报告期"]] = [pandas.datetime(2015, 6, 10), pandas.datetime(2014, 12, 31)]

In [334]:
accounting_neeq_long1.loc[(accounting_neeq_long1["证券代码"] == "832500.OC") & 
                          (accounting_neeq_long1["年度"] == 2014), 
                          ["实际披露日期", "报告期"]] = [pandas.datetime(2015, 6, 4), pandas.datetime(2014, 12, 31)]

In [335]:
accounting_neeq_long1.loc[(accounting_neeq_long1["证券代码"] == "832497.OC") & 
                          (accounting_neeq_long1["年度"] == 2014), 
                          "报告期"] = pandas.datetime(2014, 12, 31)

In [336]:
accounting_neeq_long1.loc[(accounting_neeq_long1["证券代码"] == "832496.OC") & 
                          (accounting_neeq_long1["年度"] == 2014), 
                          ["实际披露日期", "报告期"]] = [pandas.datetime(2015, 6, 4), pandas.datetime(2014, 12, 31)]

In [337]:
accounting_neeq_long1.loc[(accounting_neeq_long1["证券代码"] == "832495.OC") & 
                          (accounting_neeq_long1["年度"] == 2014), 
                          ["实际披露日期", "报告期"]] = [pandas.datetime(2015, 5, 14), pandas.datetime(2014, 12, 31)]

In [338]:
accounting_neeq_long1.loc[(accounting_neeq_long1["证券代码"] == "832494.OC") & 
                          (accounting_neeq_long1["年度"] == 2014), 
                          ["实际披露日期", "报告期"]] = [pandas.datetime(2015, 6, 19), pandas.datetime(2014, 12, 31)]

In [339]:
accounting_neeq_long1.loc[(accounting_neeq_long1["证券代码"] == "832493.OC") & 
                          (accounting_neeq_long1["年度"] == 2014), 
                          ["实际披露日期", "报告期"]] = [pandas.datetime(2015, 6, 2), pandas.datetime(2014, 12, 31)]

In [340]:
accounting_neeq_long1.loc[(accounting_neeq_long1["证券代码"] == "832492.OC") & 
                          (accounting_neeq_long1["年度"] == 2014), 
                          ["实际披露日期", "报告期"]] = [pandas.datetime(2015, 6, 25), pandas.datetime(2014, 12, 31)]

In [341]:
accounting_neeq_long1.loc[(accounting_neeq_long1["证券代码"] == "832491.OC") & 
                          (accounting_neeq_long1["年度"] == 2014), 
                          ["实际披露日期", "报告期"]] = [pandas.datetime(2015, 5, 15), pandas.datetime(2014, 12, 31)]

In [342]:
accounting_neeq_long1.loc[(accounting_neeq_long1["证券代码"] == "832489.OC") & 
                          (accounting_neeq_long1["年度"] == 2014), 
                          ["实际披露日期", "报告期"]] = [pandas.datetime(2015, 5, 26), pandas.datetime(2014, 12, 31)]

In [343]:
accounting_neeq_long1.loc[(accounting_neeq_long1["证券代码"] == "832488.OC") & 
                          (accounting_neeq_long1["年度"] == 2014), 
                          ["实际披露日期", "报告期"]] = [pandas.datetime(2015, 5, 29), pandas.datetime(2014, 12, 31)]

In [344]:
accounting_neeq_long1.loc[(accounting_neeq_long1["证券代码"] == "832487.OC") & 
                          (accounting_neeq_long1["年度"] == 2014), 
                          ["实际披露日期", "报告期"]] = [pandas.datetime(2015, 6, 19), pandas.datetime(2014, 12, 31)]

In [345]:
accounting_neeq_long1.loc[(accounting_neeq_long1["证券代码"] == "832486.OC") & 
                          (accounting_neeq_long1["年度"] == 2014), 
                          ["实际披露日期", "报告期"]] = [pandas.datetime(2015, 6, 2), pandas.datetime(2014, 12, 31)]

In [346]:
accounting_neeq_long1.loc[(accounting_neeq_long1["证券代码"] == "832485.OC") & 
                          (accounting_neeq_long1["年度"] == 2014), 
                          ["实际披露日期", "报告期"]] = [pandas.datetime(2015, 5, 18), pandas.datetime(2014, 12, 31)]

In [347]:
accounting_neeq_long1.loc[(accounting_neeq_long1["证券代码"] == "832483.OC") & 
                          (accounting_neeq_long1["年度"] == 2014), 
                          ["实际披露日期", "报告期"]] = [pandas.datetime(2015, 6, 9), pandas.datetime(2014, 12, 31)]

In [348]:
accounting_neeq_long1.loc[(accounting_neeq_long1["证券代码"] == "832481.OC") & 
                          (accounting_neeq_long1["年度"] == 2014), 
                          ["实际披露日期", "报告期"]] = [pandas.datetime(2015, 6, 9), pandas.datetime(2014, 12, 31)]

In [349]:
accounting_neeq_long1.loc[(accounting_neeq_long1["证券代码"] == "832480.OC") & 
                          (accounting_neeq_long1["年度"] == 2014), 
                          ["实际披露日期", "报告期"]] = [pandas.datetime(2015, 6, 5), pandas.datetime(2014, 12, 31)]

In [350]:
accounting_neeq_long1.loc[(accounting_neeq_long1["证券代码"] == "832479.OC") & 
                          (accounting_neeq_long1["年度"] == 2014), 
                          ["实际披露日期", "报告期"]] = [pandas.datetime(2015, 5, 27), pandas.datetime(2014, 12, 31)]

In [351]:
accounting_neeq_long1.loc[(accounting_neeq_long1["证券代码"] == "832478.OC") & 
                          (accounting_neeq_long1["年度"] == 2014), 
                          ["实际披露日期", "报告期"]] = [pandas.datetime(2015, 6, 8), pandas.datetime(2014, 12, 31)]

In [352]:
accounting_neeq_long1.loc[(accounting_neeq_long1["证券代码"] == "832477.OC") & 
                          (accounting_neeq_long1["年度"] == 2014), 
                          ["实际披露日期", "报告期"]] = [pandas.datetime(2015, 6, 10), pandas.datetime(2014, 12, 31)]

In [353]:
accounting_neeq_long1.loc[(accounting_neeq_long1["证券代码"] == "832476.OC") & 
                          (accounting_neeq_long1["年度"] == 2014), 
                          ["实际披露日期", "报告期"]] = [pandas.datetime(2015, 5, 25), pandas.datetime(2014, 12, 31)]

In [354]:
accounting_neeq_long1.loc[(accounting_neeq_long1["证券代码"] == "832475.OC") & 
                          (accounting_neeq_long1["年度"] == 2014), 
                          ["实际披露日期", "报告期"]] = [pandas.datetime(2015, 6, 5), pandas.datetime(2014, 12, 31)]

In [355]:
accounting_neeq_long1.loc[(accounting_neeq_long1["证券代码"] == "832473.OC") & 
                          (accounting_neeq_long1["年度"] == 2014), 
                          ["实际披露日期", "报告期"]] = [pandas.datetime(2015, 5, 28), pandas.datetime(2014, 12, 31)]

In [356]:
accounting_neeq_long1.loc[(accounting_neeq_long1["证券代码"] == "832472.OC") & 
                          (accounting_neeq_long1["年度"] == 2014), 
                          ["实际披露日期", "报告期"]] = [pandas.datetime(2015, 5, 27), pandas.datetime(2014, 12, 31)]

In [357]:
accounting_neeq_long1.loc[(accounting_neeq_long1["证券代码"] == "832471.OC") & 
                          (accounting_neeq_long1["年度"] == 2014), 
                          ["实际披露日期", "报告期"]] = [pandas.datetime(2015, 5, 27), pandas.datetime(2014, 12, 31)]

In [358]:
accounting_neeq_long1.loc[(accounting_neeq_long1["证券代码"] == "832470.OC") & 
                          (accounting_neeq_long1["年度"] == 2014), 
                          ["实际披露日期", "报告期"]] = [pandas.datetime(2015, 6, 5), pandas.datetime(2014, 12, 31)]

In [359]:
accounting_neeq_long1.loc[(accounting_neeq_long1["证券代码"] == "832469.OC") & 
                          (accounting_neeq_long1["年度"] == 2014), 
                          ["实际披露日期", "报告期"]] = [pandas.datetime(2015, 5, 26), pandas.datetime(2014, 12, 31)]

In [360]:
accounting_neeq_long1.loc[(accounting_neeq_long1["证券代码"] == "832468.OC") & 
                          (accounting_neeq_long1["年度"] == 2014), 
                          ["实际披露日期", "报告期"]] = [pandas.datetime(2015, 5, 14), pandas.datetime(2014, 12, 31)]

In [361]:
accounting_neeq_long1.loc[(accounting_neeq_long1["证券代码"] == "832467.OC") & 
                          (accounting_neeq_long1["年度"] == 2014), 
                          ["实际披露日期", "报告期"]] = [pandas.datetime(2015, 5, 21), pandas.datetime(2014, 12, 31)]

In [362]:
accounting_neeq_long1.loc[(accounting_neeq_long1["证券代码"] == "832466.OC") & 
                          (accounting_neeq_long1["年度"] == 2014), 
                          ["实际披露日期", "报告期"]] = [pandas.datetime(2015, 5, 18), pandas.datetime(2014, 12, 31)]

In [363]:
accounting_neeq_long1.loc[(accounting_neeq_long1["证券代码"] == "832465.OC") & 
                          (accounting_neeq_long1["年度"] == 2014), 
                          ["实际披露日期", "报告期"]] = [pandas.datetime(2015, 6, 9), pandas.datetime(2014, 12, 31)]

In [364]:
accounting_neeq_long1.loc[(accounting_neeq_long1["证券代码"] == "832464.OC") & 
                          (accounting_neeq_long1["年度"] == 2014), 
                          ["实际披露日期", "报告期"]] = [pandas.datetime(2015, 6, 9), pandas.datetime(2014, 12, 31)]

In [365]:
accounting_neeq_long1.loc[(accounting_neeq_long1["证券代码"] == "832463.OC") & 
                          (accounting_neeq_long1["年度"] == 2014), 
                          ["实际披露日期", "报告期"]] = [pandas.datetime(2015, 5, 18), pandas.datetime(2014, 12, 31)]

In [366]:
accounting_neeq_long1.loc[(accounting_neeq_long1["证券代码"] == "832462.OC") & 
                          (accounting_neeq_long1["年度"] == 2014), 
                          ["实际披露日期", "报告期"]] = [pandas.datetime(2015, 5, 13), pandas.datetime(2014, 12, 31)]

In [367]:
accounting_neeq_long1.loc[(accounting_neeq_long1["证券代码"] == "832461.OC") & 
                          (accounting_neeq_long1["年度"] == 2014), 
                          ["实际披露日期", "报告期"]] = [pandas.datetime(2015, 6, 10), pandas.datetime(2014, 12, 31)]

In [368]:
accounting_neeq_long1.loc[(accounting_neeq_long1["证券代码"] == "832457.OC") & 
                          (accounting_neeq_long1["年度"] == 2014), 
                          ["实际披露日期", "报告期"]] = [pandas.datetime(2015, 5, 29), pandas.datetime(2014, 12, 31)]

In [369]:
accounting_neeq_long1.loc[(accounting_neeq_long1["证券代码"] == "832456.OC") & 
                          (accounting_neeq_long1["年度"] == 2014), 
                          ["实际披露日期", "报告期"]] = [pandas.datetime(2015, 6, 8), pandas.datetime(2014, 12, 31)]

In [370]:
accounting_neeq_long1.loc[(accounting_neeq_long1["证券代码"] == "832454.OC") & 
                          (accounting_neeq_long1["年度"] == 2014), 
                          ["实际披露日期", "报告期"]] = [pandas.datetime(2015, 6, 3), pandas.datetime(2014, 12, 31)]

In [371]:
accounting_neeq_long1.loc[(accounting_neeq_long1["证券代码"] == "832453.OC") & 
                          (accounting_neeq_long1["年度"] == 2014), 
                          ["实际披露日期", "报告期"]] = [pandas.datetime(2015, 6, 9), pandas.datetime(2014, 12, 31)]

In [372]:
accounting_neeq_long1.loc[(accounting_neeq_long1["证券代码"] == "832452.OC") & 
                          (accounting_neeq_long1["年度"] == 2014), 
                          ["实际披露日期", "报告期"]] = [pandas.datetime(2015, 6, 10), pandas.datetime(2014, 12, 31)]

In [373]:
accounting_neeq_long1.loc[(accounting_neeq_long1["证券代码"] == "832449.OC") & 
                          (accounting_neeq_long1["年度"] == 2014), 
                          ["实际披露日期", "报告期"]] = [pandas.datetime(2015, 5, 18), pandas.datetime(2014, 12, 31)]

In [374]:
accounting_neeq_long1.loc[(accounting_neeq_long1["证券代码"] == "832448.OC") & 
                          (accounting_neeq_long1["年度"] == 2014), 
                          ["实际披露日期", "报告期"]] = [pandas.datetime(2015, 6, 11), pandas.datetime(2014, 12, 31)]

In [375]:
accounting_neeq_long1.loc[(accounting_neeq_long1["证券代码"] == "832447.OC") & 
                          (accounting_neeq_long1["年度"] == 2014), 
                          ["实际披露日期", "报告期"]] = [pandas.datetime(2015, 5, 21), pandas.datetime(2014, 12, 31)]

In [376]:
accounting_neeq_long1.loc[(accounting_neeq_long1["证券代码"] == "832446.OC") & 
                          (accounting_neeq_long1["年度"] == 2014), 
                          ["实际披露日期", "报告期"]] = [pandas.datetime(2015, 6, 19), pandas.datetime(2014, 12, 31)]

In [377]:
accounting_neeq_long1.loc[(accounting_neeq_long1["证券代码"] == "832445.OC") & 
                          (accounting_neeq_long1["年度"] == 2014), 
                          ["实际披露日期", "报告期"]] = [pandas.datetime(2015, 5, 21), pandas.datetime(2014, 12, 31)]

In [378]:
accounting_neeq_long1.loc[(accounting_neeq_long1["证券代码"] == "832444.OC") & 
                          (accounting_neeq_long1["年度"] == 2014), 
                          ["实际披露日期", "报告期"]] = [pandas.datetime(2015, 5, 11), pandas.datetime(2014, 12, 31)]

In [379]:
accounting_neeq_long1.loc[(accounting_neeq_long1["证券代码"] == "832443.OC") & 
                          (accounting_neeq_long1["年度"] == 2014), 
                          ["实际披露日期", "报告期"]] = [pandas.datetime(2015, 6, 3), pandas.datetime(2014, 12, 31)]

accounting_neeq_long1.loc[(accounting_neeq_long1["证券代码"] == "832442.OC") & 
                          (accounting_neeq_long1["年度"] == 2014), 
                          ["实际披露日期", "报告期"]] = [pandas.datetime(2015, 5, 12), pandas.datetime(2014, 12, 31)]

accounting_neeq_long1.loc[(accounting_neeq_long1["证券代码"] == "832440.OC") & 
                          (accounting_neeq_long1["年度"] == 2014), 
                          ["实际披露日期", "报告期"]] = [pandas.datetime(2015, 5, 25), pandas.datetime(2014, 12, 31)]

accounting_neeq_long1.loc[(accounting_neeq_long1["证券代码"] == "832439.OC") & 
                          (accounting_neeq_long1["年度"] == 2014), 
                          ["实际披露日期", "报告期"]] = [pandas.datetime(2015, 5, 28), pandas.datetime(2014, 12, 31)]

accounting_neeq_long1.loc[(accounting_neeq_long1["证券代码"] == "832438.OC") & 
                          (accounting_neeq_long1["年度"] == 2014), 
                          ["实际披露日期", "报告期"]] = [pandas.datetime(2015, 6, 9), pandas.datetime(2014, 12, 31)]

accounting_neeq_long1.loc[(accounting_neeq_long1["证券代码"] == "832437.OC") & 
                          (accounting_neeq_long1["年度"] == 2014), 
                          ["实际披露日期", "报告期"]] = [pandas.datetime(2015, 6, 16), pandas.datetime(2014, 12, 31)]

accounting_neeq_long1.loc[(accounting_neeq_long1["证券代码"] == "832436.OC") & 
                          (accounting_neeq_long1["年度"] == 2014), 
                          ["实际披露日期", "报告期"]] = [pandas.datetime(2015, 5, 28), pandas.datetime(2014, 12, 31)]

accounting_neeq_long1.loc[(accounting_neeq_long1["证券代码"] == "832434.OC") & 
                          (accounting_neeq_long1["年度"] == 2014), 
                          ["实际披露日期", "报告期"]] = [pandas.datetime(2015, 5, 29), pandas.datetime(2014, 12, 31)]

accounting_neeq_long1.loc[(accounting_neeq_long1["证券代码"] == "832433.OC") & 
                          (accounting_neeq_long1["年度"] == 2014), 
                          ["实际披露日期", "报告期"]] = [pandas.datetime(2015, 5, 28), pandas.datetime(2014, 12, 31)]

accounting_neeq_long1.loc[(accounting_neeq_long1["证券代码"] == "832432.OC") & 
                          (accounting_neeq_long1["年度"] == 2014), 
                          ["实际披露日期", "报告期"]] = [pandas.datetime(2015, 7, 14), pandas.datetime(2014, 12, 31)]

accounting_neeq_long1.loc[(accounting_neeq_long1["证券代码"] == "832427.OC") & 
                          (accounting_neeq_long1["年度"] == 2014), 
                          ["实际披露日期", "报告期"]] = [pandas.datetime(2015, 6, 5), pandas.datetime(2014, 12, 31)]

accounting_neeq_long1.loc[(accounting_neeq_long1["证券代码"] == "832426.OC") & 
                          (accounting_neeq_long1["年度"] == 2014), 
                          ["实际披露日期", "报告期"]] = [pandas.datetime(2015, 6, 1), pandas.datetime(2014, 12, 31)]

accounting_neeq_long1.loc[(accounting_neeq_long1["证券代码"] == "832425.OC") & 
                          (accounting_neeq_long1["年度"] == 2014), 
                          ["实际披露日期", "报告期"]] = [pandas.datetime(2015, 5, 29), pandas.datetime(2014, 12, 31)]

accounting_neeq_long1.loc[(accounting_neeq_long1["证券代码"] == "832424.OC") & 
                          (accounting_neeq_long1["年度"] == 2014), 
                          ["实际披露日期", "报告期"]] = [pandas.datetime(2015, 10, 26), pandas.datetime(2014, 12, 31)]

accounting_neeq_long1.loc[(accounting_neeq_long1["证券代码"] == "832422.OC") & 
                          (accounting_neeq_long1["年度"] == 2014), 
                          ["实际披露日期", "报告期"]] = [pandas.datetime(2015, 5, 12), pandas.datetime(2014, 12, 31)]

accounting_neeq_long1.loc[(accounting_neeq_long1["证券代码"] == "832421.OC") & 
                          (accounting_neeq_long1["年度"] == 2014), 
                          ["实际披露日期", "报告期"]] = [pandas.datetime(2015, 5, 22), pandas.datetime(2014, 12, 31)]

accounting_neeq_long1.loc[(accounting_neeq_long1["证券代码"] == "832420.OC") & 
                          (accounting_neeq_long1["年度"] == 2014), 
                          ["实际披露日期", "报告期"]] = [pandas.datetime(2015, 6, 9), pandas.datetime(2014, 12, 31)]

accounting_neeq_long1.loc[(accounting_neeq_long1["证券代码"] == "832418.OC") & 
                          (accounting_neeq_long1["年度"] == 2014), 
                          ["实际披露日期", "报告期"]] = [pandas.datetime(2015, 5, 18), pandas.datetime(2014, 12, 31)]

accounting_neeq_long1.loc[(accounting_neeq_long1["证券代码"] == "832417.OC") & 
                          (accounting_neeq_long1["年度"] == 2014), 
                          ["实际披露日期", "报告期"]] = [pandas.datetime(2015, 6, 5), pandas.datetime(2014, 12, 31)]

accounting_neeq_long1.loc[(accounting_neeq_long1["证券代码"] == "832416.OC") & 
                          (accounting_neeq_long1["年度"] == 2014), 
                          ["实际披露日期", "报告期"]] = [pandas.datetime(2015, 5, 19), pandas.datetime(2014, 12, 31)]

accounting_neeq_long1.loc[(accounting_neeq_long1["证券代码"] == "832415.OC") & 
                          (accounting_neeq_long1["年度"] == 2014), 
                          ["实际披露日期", "报告期"]] = [pandas.datetime(2015, 6, 5), pandas.datetime(2014, 12, 31)]

accounting_neeq_long1.loc[(accounting_neeq_long1["证券代码"] == "832414.OC") & 
                          (accounting_neeq_long1["年度"] == 2014), 
                          ["实际披露日期", "报告期"]] = [pandas.datetime(2015, 5, 27), pandas.datetime(2014, 12, 31)]

accounting_neeq_long1.loc[(accounting_neeq_long1["证券代码"] == "832413.OC") & 
                          (accounting_neeq_long1["年度"] == 2014), 
                          ["实际披露日期", "报告期"]] = [pandas.datetime(2015, 5, 19), pandas.datetime(2014, 12, 31)]

accounting_neeq_long1.loc[(accounting_neeq_long1["证券代码"] == "832412.OC") & 
                          (accounting_neeq_long1["年度"] == 2014), 
                          ["实际披露日期", "报告期"]] = [pandas.datetime(2015, 5, 12), pandas.datetime(2014, 12, 31)]

accounting_neeq_long1.loc[(accounting_neeq_long1["证券代码"] == "832411.OC") & 
                          (accounting_neeq_long1["年度"] == 2014), 
                          ["实际披露日期", "报告期"]] = [pandas.datetime(2015, 5, 28), pandas.datetime(2014, 12, 31)]

accounting_neeq_long1.loc[(accounting_neeq_long1["证券代码"] == "832410.OC") & 
                          (accounting_neeq_long1["年度"] == 2014), 
                          ["实际披露日期", "报告期"]] = [pandas.datetime(2015, 5, 19), pandas.datetime(2014, 12, 31)]

accounting_neeq_long1.loc[(accounting_neeq_long1["证券代码"] == "832409.OC") & 
                          (accounting_neeq_long1["年度"] == 2014), 
                          ["实际披露日期", "报告期"]] = [pandas.datetime(2015, 5, 5), pandas.datetime(2014, 12, 31)]

accounting_neeq_long1.loc[(accounting_neeq_long1["证券代码"] == "832407.OC") & 
                          (accounting_neeq_long1["年度"] == 2014), 
                          ["实际披露日期", "报告期"]] = [pandas.datetime(2015, 6, 1), pandas.datetime(2014, 12, 31)]

accounting_neeq_long1.loc[(accounting_neeq_long1["证券代码"] == "832406.OC") & 
                          (accounting_neeq_long1["年度"] == 2014), 
                          ["实际披露日期", "报告期"]] = [pandas.datetime(2015, 5, 21), pandas.datetime(2014, 12, 31)]

accounting_neeq_long1.loc[(accounting_neeq_long1["证券代码"] == "832404.OC") & 
                          (accounting_neeq_long1["年度"] == 2014), 
                          ["实际披露日期", "报告期"]] = [pandas.datetime(2015, 5, 12), pandas.datetime(2014, 12, 31)]

accounting_neeq_long1.loc[(accounting_neeq_long1["证券代码"] == "832403.OC") & 
                          (accounting_neeq_long1["年度"] == 2014), 
                          ["实际披露日期", "报告期"]] = [pandas.datetime(2015, 5, 8), pandas.datetime(2014, 12, 31)]

accounting_neeq_long1.loc[(accounting_neeq_long1["证券代码"] == "832398.OC") & 
                          (accounting_neeq_long1["年度"] == 2014), 
                          ["实际披露日期", "报告期"]] = [pandas.datetime(2015, 5, 7), pandas.datetime(2014, 12, 31)]

accounting_neeq_long1.loc[(accounting_neeq_long1["证券代码"] == "832395.OC") & 
                          (accounting_neeq_long1["年度"] == 2014), 
                          ["实际披露日期", "报告期"]] = [pandas.datetime(2015, 5, 12), pandas.datetime(2014, 12, 31)]

accounting_neeq_long1.loc[(accounting_neeq_long1["证券代码"] == "832394.OC") & 
                          (accounting_neeq_long1["年度"] == 2014), 
                          ["实际披露日期", "报告期"]] = [pandas.datetime(2015, 5, 8), pandas.datetime(2014, 12, 31)]

accounting_neeq_long1.loc[(accounting_neeq_long1["证券代码"] == "832393.OC") & 
                          (accounting_neeq_long1["年度"] == 2014), 
                          ["实际披露日期", "报告期"]] = [pandas.datetime(2015, 6, 18), pandas.datetime(2014, 12, 31)]

accounting_neeq_long1.loc[(accounting_neeq_long1["证券代码"] == "832391.OC") & 
                          (accounting_neeq_long1["年度"] == 2014), 
                          ["实际披露日期", "报告期"]] = [pandas.datetime(2015, 5, 20), pandas.datetime(2014, 12, 31)]

accounting_neeq_long1.loc[(accounting_neeq_long1["证券代码"] == "832390.OC") & 
                          (accounting_neeq_long1["年度"] == 2014), 
                          ["实际披露日期", "报告期"]] = [pandas.datetime(2015, 5, 5), pandas.datetime(2014, 12, 31)]

accounting_neeq_long1.loc[(accounting_neeq_long1["证券代码"] == "832389.OC") & 
                          (accounting_neeq_long1["年度"] == 2014), 
                          ["实际披露日期", "报告期"]] = [pandas.datetime(2015, 5, 6), pandas.datetime(2014, 12, 31)]

accounting_neeq_long1.loc[(accounting_neeq_long1["证券代码"] == "832388.OC") & 
                          (accounting_neeq_long1["年度"] == 2014), 
                          ["实际披露日期", "报告期"]] = [pandas.datetime(2015, 6, 4), pandas.datetime(2014, 12, 31)]


accounting_neeq_long1.loc[(accounting_neeq_long1["证券代码"] == "832387.OC") & 
                          (accounting_neeq_long1["年度"] == 2014), 
                          ["实际披露日期", "报告期"]] = [pandas.datetime(2015, 6, 8), pandas.datetime(2014, 12, 31)]

accounting_neeq_long1.loc[(accounting_neeq_long1["证券代码"] == "832386.OC") & 
                          (accounting_neeq_long1["年度"] == 2014), 
                          ["实际披露日期", "报告期"]] = [pandas.datetime(2015, 5, 18), pandas.datetime(2014, 12, 31)]

accounting_neeq_long1.loc[(accounting_neeq_long1["证券代码"] == "832377.OC") & 
                          (accounting_neeq_long1["年度"] == 2014), 
                          ["实际披露日期", "报告期"]] = [pandas.datetime(2015, 5, 11), pandas.datetime(2014, 12, 31)]

accounting_neeq_long1.loc[(accounting_neeq_long1["证券代码"] == "832374.OC") & 
                          (accounting_neeq_long1["年度"] == 2014), 
                          ["实际披露日期", "报告期"]] = [pandas.datetime(2015, 6, 12), pandas.datetime(2014, 12, 31)]

accounting_neeq_long1.loc[(accounting_neeq_long1["证券代码"] == "832373.OC") & 
                          (accounting_neeq_long1["年度"] == 2014), 
                          ["实际披露日期", "报告期"]] = [pandas.datetime(2015, 5, 27), pandas.datetime(2014, 12, 31)]

accounting_neeq_long1.loc[(accounting_neeq_long1["证券代码"] == "832360.OC") & 
                          (accounting_neeq_long1["年度"] == 2014), 
                          ["实际披露日期", "报告期"]] = [pandas.datetime(2015, 5, 20), pandas.datetime(2014, 12, 31)]

accounting_neeq_long1.loc[(accounting_neeq_long1["证券代码"] == "832355.OC") & 
                          (accounting_neeq_long1["年度"] == 2014), 
                          ["实际披露日期", "报告期"]] = [pandas.datetime(2015, 5, 29), pandas.datetime(2014, 12, 31)]

accounting_neeq_long1.loc[(accounting_neeq_long1["证券代码"] == "832347.OC") & 
                          (accounting_neeq_long1["年度"] == 2014), 
                          ["实际披露日期", "报告期"]] = [pandas.datetime(2015, 4, 29), pandas.datetime(2014, 12, 31)]

accounting_neeq_long1.loc[(accounting_neeq_long1["证券代码"] == "830819.OC") & 
                          (accounting_neeq_long1["年度"] == 2013), 
                          ["实际披露日期", "报告期"]] = [pandas.datetime(2014, 8, 6), pandas.datetime(2013, 12, 31)]

accounting_neeq_long1.loc[(accounting_neeq_long1["证券代码"] == "832337.OC") & 
                          (accounting_neeq_long1["年度"] == 2014), 
                          ["实际披露日期", "报告期"]] = [pandas.datetime(2015, 5, 12), pandas.datetime(2014, 12, 31)]

accounting_neeq_long1.loc[(accounting_neeq_long1["证券代码"] == "830800.OC") & 
                          (accounting_neeq_long1["年度"] == 2013), 
                          ["实际披露日期", "报告期"]] = [pandas.datetime(2014, 5, 30), pandas.datetime(2013, 12, 31)]

accounting_neeq_long1.loc[(accounting_neeq_long1["证券代码"] == "830799.OC") & 
                          (accounting_neeq_long1["年度"] == 2013), 
                          ["实际披露日期", "报告期"]] = [pandas.datetime(2014, 6, 27), pandas.datetime(2013, 12, 31)]

accounting_neeq_long1.loc[(accounting_neeq_long1["证券代码"] == "830786.OC") & 
                          (accounting_neeq_long1["年度"] == 2013), 
                          ["实际披露日期", "报告期"]] = [pandas.datetime(2014, 5, 30), pandas.datetime(2013, 12, 31)]

accounting_neeq_long1.loc[(accounting_neeq_long1["证券代码"] == "830790.OC") & 
                          (accounting_neeq_long1["年度"] == 2013), 
                          ["实际披露日期", "报告期"]] = [pandas.datetime(2014, 7, 7), pandas.datetime(2013, 12, 31)]

accounting_neeq_long1.loc[(accounting_neeq_long1["证券代码"] == "830787.OC") & 
                          (accounting_neeq_long1["年度"] == 2013), 
                          ["实际披露日期", "报告期"]] = [pandas.datetime(2014, 6, 30), pandas.datetime(2013, 12, 31)]

accounting_neeq_long1.loc[(accounting_neeq_long1["证券代码"] == "830785.OC") & 
                          (accounting_neeq_long1["年度"] == 2013), 
                          ["实际披露日期", "报告期"]] = [pandas.datetime(2014, 6, 11), pandas.datetime(2013, 12, 31)]

In [380]:
accounting_neeq_long1.loc[(accounting_neeq_long1["证券代码"] == "832482.OC") & 
                          (accounting_neeq_long1["年度"] == 2014), 
                          ["实际披露日期", "报告期"]] = [pandas.datetime(2015, 5, 18), pandas.datetime(2014, 12, 31)]

In [381]:
accounting_neeq_long1.loc[(accounting_neeq_long1["证券代码"] == "832498.OC") & 
                          (accounting_neeq_long1["年度"] == 2014), 
                          ["实际披露日期", "报告期"]] = [pandas.datetime(2015, 6, 23), pandas.datetime(2014, 12, 31)]

In [382]:
accounting_neeq_long1.loc[(accounting_neeq_long1["证券代码"] == "832537.OC") & 
                          (accounting_neeq_long1["年度"] == 2014), 
                          ["实际披露日期", "报告期"]] = [pandas.datetime(2015, 6, 5), pandas.datetime(2014, 12, 31)]

In [383]:
accounting_neeq_long1.loc[(accounting_neeq_long1["证券代码"] == "832568.OC") & 
                          (accounting_neeq_long1["年度"] == 2015), 
                          ["折旧", "长期待摊费用摊销", "存货的减少", "经营性应收项目的减少", "经营性应付项目的增加"]] =\
[14500391.74, 0, 32365482.17, -102364408.41, -137055152.65]

accounting_neeq_long1.loc[(accounting_neeq_long1["证券代码"] == "832568.OC") & 
                          (accounting_neeq_long1["年度"] == 2014), 
                          ["折旧", "长期待摊费用摊销", "存货的减少", "经营性应收项目的减少", "经营性应付项目的增加"]] =\
[10058905.53, 0, -17061012.49,  -10599958.09, 459435.69]

accounting_neeq_long1.loc[(accounting_neeq_long1["证券代码"] == "832568.OC") & 
                          (accounting_neeq_long1["年度"] == 2013), 
                          ["折旧", "长期待摊费用摊销", "存货的减少", "经营性应收项目的减少", "经营性应付项目的增加"]] =\
[11218661.06, 0, -25707153.13, 34482646.00, 112376137.46]

In [384]:
accounting_neeq_long1.loc[(accounting_neeq_long1["证券代码"] == "430702.OC") &
                          (accounting_neeq_long1["年度"] == 2013), ["实际披露日期", "报告期"]] =\
[pandas.datetime(2014, 5, 21), pandas.datetime(2013, 12, 31)]

In [385]:
accounting_neeq_long1.loc[(accounting_neeq_long1["证券代码"] == "830825.OC") &
                          (accounting_neeq_long1["年度"] == 2013), ["实际披露日期", "报告期"]] =\
[pandas.datetime(2014, 6, 24), pandas.datetime(2013, 12, 31)]

In [386]:
accounting_neeq_long1.loc[(accounting_neeq_long1["证券代码"] == "830767.OC") &
                          (accounting_neeq_long1["年度"] == 2013), ["实际披露日期", "报告期"]] =\
[pandas.datetime(2014, 6, 9), pandas.datetime(2013, 12, 31)]

In [387]:
accounting_neeq_long1.loc[(accounting_neeq_long1["证券代码"] == "830780.OC") &
                          (accounting_neeq_long1["年度"] == 2013), ["实际披露日期", "报告期"]] =\
[pandas.datetime(2014, 6, 9), pandas.datetime(2013, 12, 31)]

In [388]:
accounting_neeq_long1.loc[(accounting_neeq_long1["证券代码"] == "832383.OC") &
                          (accounting_neeq_long1["年度"] == 2014), ["实际披露日期", "报告期"]] =\
[pandas.datetime(2015, 6, 9), pandas.datetime(2014, 12, 31)]

In [389]:
accounting_neeq_long1.loc[(accounting_neeq_long1["证券代码"] == "832611.OC") &
                          (accounting_neeq_long1["年度"] == 2014), ["实际披露日期", "报告期"]] =\
[pandas.datetime(2015, 6, 11), pandas.datetime(2014, 12, 31)]

In [390]:
accounting_neeq_long1.loc[(accounting_neeq_long1["证券代码"] == "832615.OC") &
                          (accounting_neeq_long1["年度"] == 2014), ["实际披露日期", "报告期"]] =\
[pandas.datetime(2015, 7, 17), pandas.datetime(2014, 12, 31)]

In [391]:
accounting_neeq_long1.loc[(accounting_neeq_long1["证券代码"] == "832686.OC") &
                          (accounting_neeq_long1["年度"] == 2014), ["实际披露日期", "报告期"]] =\
[pandas.datetime(2015, 10, 13), pandas.datetime(2014, 12, 31)]

In [392]:
accounting_neeq_long1.loc[(accounting_neeq_long1["证券代码"] == "430699.OC") &
                          (accounting_neeq_long1["年度"] == 2013), ["实际披露日期", "报告期"]] =\
[pandas.datetime(2014, 5, 20), pandas.datetime(2013, 12, 31)]

In [393]:
accounting_neeq_long1.loc[(accounting_neeq_long1["证券代码"] == "430718.OC") &
                          (accounting_neeq_long1["年度"] == 2013), ["实际披露日期", "报告期"]] =\
[pandas.datetime(2014, 5, 14), pandas.datetime(2013, 12, 31)]

In [394]:
accounting_neeq_long1.loc[(accounting_neeq_long1["证券代码"] == "430720.OC") &
                          (accounting_neeq_long1["年度"] == 2013), ["实际披露日期", "报告期"]] =\
[pandas.datetime(2014, 5, 7), pandas.datetime(2013, 12, 31)]

In [395]:
accounting_neeq_long1.loc[(accounting_neeq_long1["证券代码"] == "430722.OC") &
                          (accounting_neeq_long1["年度"] == 2013), ["实际披露日期", "报告期"]] =\
[pandas.datetime(2014, 5, 23), pandas.datetime(2013, 12, 31)]

In [396]:
accounting_neeq_long1.loc[(accounting_neeq_long1["证券代码"] == "430725.OC") &
                          (accounting_neeq_long1["年度"] == 2013), ["实际披露日期", "报告期"]] =\
[pandas.datetime(2014, 5, 21), pandas.datetime(2013, 12, 31)]

In [397]:
accounting_neeq_long1.loc[(accounting_neeq_long1["证券代码"] == "430727.OC") &
                          (accounting_neeq_long1["年度"] == 2013), ["实际披露日期", "报告期"]] =\
[pandas.datetime(2014, 6, 4), pandas.datetime(2013, 12, 31)]

In [398]:
accounting_neeq_long1.loc[(accounting_neeq_long1["证券代码"] == "430729.OC") &
                          (accounting_neeq_long1["年度"] == 2013), ["实际披露日期", "报告期"]] =\
[pandas.datetime(2014, 5, 30), pandas.datetime(2013, 12, 31)]

In [399]:
accounting_neeq_long1.loc[(accounting_neeq_long1["证券代码"] == "430730.OC") &
                          (accounting_neeq_long1["年度"] == 2013), ["实际披露日期", "报告期"]] =\
[pandas.datetime(2014, 5, 16), pandas.datetime(2013, 12, 31)]

In [400]:
accounting_neeq_long1.loc[(accounting_neeq_long1["证券代码"] == "430733.OC") &
                          (accounting_neeq_long1["年度"] == 2013), ["实际披露日期", "报告期"]] =\
[pandas.datetime(2014, 5, 19), pandas.datetime(2013, 12, 31)]

In [401]:
accounting_neeq_long1.loc[(accounting_neeq_long1["证券代码"] == "430734.OC") &
                          (accounting_neeq_long1["年度"] == 2013), ["实际披露日期", "报告期"]] =\
[pandas.datetime(2014, 5, 8), pandas.datetime(2013, 12, 31)]

In [402]:
accounting_neeq_long1.loc[(accounting_neeq_long1["证券代码"] == "430735.OC") &
                          (accounting_neeq_long1["年度"] == 2013), ["实际披露日期", "报告期"]] =\
[pandas.datetime(2014, 7, 11), pandas.datetime(2013, 12, 31)]

In [403]:
accounting_neeq_long1.loc[(accounting_neeq_long1["证券代码"] == "430736.OC") &
                          (accounting_neeq_long1["年度"] == 2013), ["实际披露日期", "报告期"]] =\
[pandas.datetime(2014, 5, 8), pandas.datetime(2013, 12, 31)]

In [404]:
accounting_neeq_long1.loc[(accounting_neeq_long1["证券代码"] == "430741.OC") &
                          (accounting_neeq_long1["年度"] == 2013), ["实际披露日期", "报告期"]] =\
[pandas.datetime(2014, 5, 9), pandas.datetime(2013, 12, 31)]

In [405]:
accounting_neeq_long1.loc[(accounting_neeq_long1["证券代码"] == "430743.OC") &
                          (accounting_neeq_long1["年度"] == 2013), ["实际披露日期", "报告期"]] =\
[pandas.datetime(2014, 5, 30), pandas.datetime(2013, 12, 31)]

In [406]:
accounting_neeq_long1.loc[(accounting_neeq_long1["证券代码"] == "430744.OC") &
                          (accounting_neeq_long1["年度"] == 2013), ["实际披露日期", "报告期"]] =\
[pandas.datetime(2014, 5, 5), pandas.datetime(2013, 12, 31)]

In [407]:
accounting_neeq_long1.loc[(accounting_neeq_long1["证券代码"] == "430745.OC") &
                          (accounting_neeq_long1["年度"] == 2013), ["实际披露日期", "报告期"]] =\
[pandas.datetime(2014, 5, 26), pandas.datetime(2013, 12, 31)]

In [408]:
accounting_neeq_long1.loc[(accounting_neeq_long1["证券代码"] == "430746.OC") &
                          (accounting_neeq_long1["年度"] == 2013), ["实际披露日期", "报告期"]] =\
[pandas.datetime(2014, 6, 5), pandas.datetime(2013, 12, 31)]

In [409]:
accounting_neeq_long1.loc[(accounting_neeq_long1["证券代码"] == "430747.OC") &
                          (accounting_neeq_long1["年度"] == 2013), ["实际披露日期", "报告期"]] =\
[pandas.datetime(2014, 5, 20), pandas.datetime(2013, 12, 31)]

In [410]:
accounting_neeq_long1.loc[(accounting_neeq_long1["证券代码"] == "430749.OC") &
                          (accounting_neeq_long1["年度"] == 2013), ["实际披露日期", "报告期"]] =\
[pandas.datetime(2014, 5, 15), pandas.datetime(2013, 12, 31)]

In [411]:
accounting_neeq_long1.loc[(accounting_neeq_long1["证券代码"] == "430750.OC") &
                          (accounting_neeq_long1["年度"] == 2013), ["实际披露日期", "报告期"]] =\
[pandas.datetime(2014, 6, 9), pandas.datetime(2013, 12, 31)]

In [412]:
accounting_neeq_long1.loc[(accounting_neeq_long1["证券代码"] == "430751.OC") &
                          (accounting_neeq_long1["年度"] == 2013), ["实际披露日期", "报告期"]] =\
[pandas.datetime(2014, 5, 30), pandas.datetime(2013, 12, 31)]

In [413]:
accounting_neeq_long1.loc[(accounting_neeq_long1["证券代码"] == "430752.OC") &
                          (accounting_neeq_long1["年度"] == 2013), ["实际披露日期", "报告期"]] =\
[pandas.datetime(2014, 5, 30), pandas.datetime(2013, 12, 31)]

In [414]:
accounting_neeq_long1.loc[(accounting_neeq_long1["证券代码"] == "430754.OC") &
                          (accounting_neeq_long1["年度"] == 2013), ["实际披露日期", "报告期"]] =\
[pandas.datetime(2014, 6, 9), pandas.datetime(2013, 12, 31)]

In [415]:
accounting_neeq_long1.loc[(accounting_neeq_long1["证券代码"] == "430755.OC") &
                          (accounting_neeq_long1["年度"] == 2013), ["实际披露日期", "报告期"]] =\
[pandas.datetime(2014, 6, 24), pandas.datetime(2013, 12, 31)]

In [416]:
accounting_neeq_long1.loc[(accounting_neeq_long1["证券代码"] == "430757.OC") &
                          (accounting_neeq_long1["年度"] == 2013), ["实际披露日期", "报告期"]] =\
[pandas.datetime(2014, 5, 30), pandas.datetime(2013, 12, 31)]

In [417]:
accounting_neeq_long1.loc[(accounting_neeq_long1["证券代码"] == "430758.OC") &
                          (accounting_neeq_long1["年度"] == 2013), ["实际披露日期", "报告期"]] =\
[pandas.datetime(2014, 6, 6), pandas.datetime(2013, 12, 31)]

In [418]:
accounting_neeq_long1.loc[(accounting_neeq_long1["证券代码"] == "430759.OC") &
                          (accounting_neeq_long1["年度"] == 2013), ["实际披露日期", "报告期"]] =\
[pandas.datetime(2014, 6, 12), pandas.datetime(2013, 12, 31)]

In [419]:
accounting_neeq_long1.loc[(accounting_neeq_long1["证券代码"] == "430760.OC") &
                          (accounting_neeq_long1["年度"] == 2013), ["实际披露日期", "报告期"]] =\
[pandas.datetime(2014, 6, 11), pandas.datetime(2013, 12, 31)]

In [420]:
accounting_neeq_long1.loc[(accounting_neeq_long1["证券代码"] == "430761.OC") &
                          (accounting_neeq_long1["年度"] == 2013), ["实际披露日期", "报告期"]] =\
[pandas.datetime(2014, 6, 11), pandas.datetime(2013, 12, 31)]

In [421]:
accounting_neeq_long1.loc[(accounting_neeq_long1["证券代码"] == "430763.OC") &
                          (accounting_neeq_long1["年度"] == 2013), ["实际披露日期", "报告期"]] =\
[pandas.datetime(2014, 6, 3), pandas.datetime(2013, 12, 31)]

In [422]:
accounting_neeq_long1.loc[(accounting_neeq_long1["证券代码"] == "830773.OC") &
                          (accounting_neeq_long1["年度"] == 2013), ["实际披露日期", "报告期"]] =\
[pandas.datetime(2014, 6, 17), pandas.datetime(2013, 12, 31)]

In [423]:
accounting_neeq_long1.loc[(accounting_neeq_long1["证券代码"] == "830775.OC") &
                          (accounting_neeq_long1["年度"] == 2013), ["实际披露日期", "报告期"]] =\
[pandas.datetime(2014, 6, 16), pandas.datetime(2013, 12, 31)]

In [424]:
accounting_neeq_long1.loc[(accounting_neeq_long1["证券代码"] == "830778.OC") &
                          (accounting_neeq_long1["年度"] == 2013), ["实际披露日期", "报告期"]] =\
[pandas.datetime(2014, 6, 3), pandas.datetime(2013, 12, 31)]

In [425]:
accounting_neeq_long1.loc[(accounting_neeq_long1["证券代码"] == "830781.OC") &
                          (accounting_neeq_long1["年度"] == 2013), ["实际披露日期", "报告期"]] =\
[pandas.datetime(2014, 9, 25), pandas.datetime(2013, 12, 31)]

In [427]:
file = "FinancialStatementDisclosingDateUpdated.csv"
accounting_neeq_long1.loc[(accounting_neeq_long1["证券代码"].isin(pandas.Series(StkcdChangedYear, name="证券代码"))) & 
                          (accounting_neeq_long1["relativeYear"] == 0)].to_csv("../data/" + file, 
                                                                              index=False)

In [429]:
accounting_neeq_long1.to_csv("../data/NEEQAccountingData.csv", index=False)

In [430]:
accounting_neeq_long1.columns

Index(['证券代码', '年度', '挂牌日期', '主办券商', '所属园区', '所属证监会行业', '摘牌日期', '挂牌年度', '挂牌月份',
       '证券简称', '资产总计', '归属母公司股东的权益', '营业收入', '营业成本', '销售费用', '管理费用', '财务费用',
       '归属母公司股东的净利润', '非经常性损益', '扣除非经常性损益后的净利', '经营活动产生的现金流量净额', '应收账款',
       '固定资产', 'ROA', '折旧', '长期待摊费用摊销', '存货的减少', '经营性应收项目的减少', '经营性应付项目的增加',
       '实际披露日期', '报告期', '挂牌前后', 'relativeYear'],
      dtype='object')

In [ ]:
items = ["资产总计", "营业收入", "归属母公司股东的净利润", "经营活动产生的现金流量净额", "固定资产"]
accounting_neeq_long1[item].isnull().all()